In [ ]:
#| default_exp radiance_cal

# Radiance Calibration.

Radiance Calibration data and supporting code.

#| hide
# References
- [ORIEL / Newport 6333 Quartz Tungsten Halogen documentation.](https://www.newport.com/medias/sys_master/images/images/hfb/hdf/8797196451870/Light-Sources.pdf)

#| hide
## Imports

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev

In [ ]:
#| hide
# Running this cell, will regenerate the exported python code.
nbdev.nbdev_export()

In [ ]:
#| export 
import numpy  as np
import pandas as pd

The following imports are for testing code in this file and for examining the data.  
These are not exported.

In [ ]:
import panel            as pn
from bokeh.plotting import figure, show 
from bokeh.io       import show, output_notebook
output_notebook()

Loading BokehJS ...

## Functions

In [ ]:
#| export
def radcal(
  spectra:object,   # A hab_spectra object containing the y_raw spectra, wavelengths
  cal_data:object   # The calibration data object.
) ->object:         # A numpy array of radiance calibrated values.
  '''
  Calibrate a spectra for radiance.
  '''
  exp = spectra.Exposure
  y_calibrated = ( spectra.raw_y - cal_data.dark[exp]) * cal_data.scale[exp]
  return y_calibrated
  

#| hide
### def read_radiance_calibration_file()

In [ ]:
#| export
def read_radiance_calibration_file(
  wavelengths, # Numpy array of wavelengths to interpolate too.
  rad_cal_fn   # Radiance Calibration File Name
) -> object:   # Numpy array of interpolated irradiance data.
  '''
  Reads a two column csv irradiance calibration file where the first 
  column is 'Wavelength', and the second 'Irradiance'. It returns an
  interpolated numpy array that matches the the `wavelengths` parameter.
  '''
  rad_cal_pd = pd.read_csv( rad_cal_fn, sep=',', comment='#' )
  y_values = np.interp( wavelengths, rad_cal_pd['Wavelength'], rad_cal_pd['Irradiance'])
  return y_values

Note:  A correction for distance from the bulb needs to be made.

#| hide
### def radiance_distance_scale_factor()

In [ ]:
#| export
def radiance_distance_scale_factor( 
  distance_m, 
  cal_distance_m=0.5
) -> float:
  '''
  Calculate the radiance scale factor for given distance (meters) from
  the QTH bulb to the broadband reflector.
  '''
  scale = cal_distance_m**2 / distance_m**2
  return scale

Test `radiance_distance_scale_factor()`.

In [ ]:
cal_distance_m=0.5
cal_distance_sq = cal_distance_m**2.0
print("Distance    Ref_scale  Scale     Errors")
errors = 0.0

# Loop through an array of distance values
for distance_m in [ 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]:
  sq = distance_m**2.0
  ref = cal_distance_sq/sq
  rv = radiance_distance_scale_factor(distance_m)
  error = rv-ref
  errors += error
  print(f'{distance_m:8.3f}     {ref:6.3f}    {rv:5.3f}     {rv-ref:6.3f} ')

if errors >0.0:
  print(f'{errors=}')
else:
  print('\nNo errors found')

Distance    Ref_scale  Scale     Errors
   0.300      2.778    2.778      0.000 
   0.400      1.562    1.562      0.000 
   0.500      1.000    1.000      0.000 
   0.600      0.694    0.694      0.000 
   0.700      0.510    0.510      0.000 
   0.800      0.391    0.391      0.000 
   0.900      0.309    0.309      0.000 
   1.000      0.250    0.250      0.000 
   1.100      0.207    0.207      0.000 
   1.200      0.174    0.174      0.000 
   1.300      0.148    0.148      0.000 
   1.400      0.128    0.128      0.000 
   1.500      0.111    0.111      0.000 
   1.600      0.098    0.098      0.000 
   1.700      0.087    0.087      0.000 
   1.800      0.077    0.077      0.000 
   1.900      0.069    0.069      0.000 
   2.000      0.062    0.062      0.000 

No errors found


Generate an array of wavelengths for testing.  For actual use, this would be the
wavelength array from a spectra.

In [ ]:
xw = np.arange(400,881, (881-400)/1280 )
len(xw), xw

(1280,
 array([400.        , 400.37578125, 400.7515625 , ..., 879.87265625,
        880.2484375 , 880.62421875]))

Read the QTH bulb calibration file.

In [ ]:
rad_cal_filename = "./2023-0728-newport-6333-QTH.csv"
rad_cal_y = read_radiance_calibration_file(xw, rad_cal_filename )

below examine the length of the array, and the interpolated values for each wavelength.

In [ ]:
len(rad_cal_y), rad_cal_y

(1280,
 array([ 4.63324251,  4.65917627,  4.68511003, ..., 31.57101339,
        31.56940725, 31.5678011 ]))

# Testing and Reviewing the data.

## Plot the data. 

In [ ]:
# Generate the Bokeh plot widget.
TOOLTIPS = [ ("Pixel Number:", "$index"), ("Wavelength:","@x{f0.1} nm"), ("Irradiance:", "@y{f0.1} mW m⁻² nm⁻¹") ]
spectra_w = figure(       title = f"{rad_cal_filename}",
                   x_axis_label = 'Wavelengthₙₘ', 
                   y_axis_label = 'Irradiance at 0.5m  mW m⁻² nm⁻¹',
                         height = 500, 
                          width = 700,
                          tools ='pan, hover, wheel_zoom, box_zoom, undo, redo, reset', # The default set of plot tools.
                       tooltips = TOOLTIPS,
                  active_scroll = 'wheel_zoom',
                 output_backend ="webgl",                # Use webGL enhancement as default.
                  )

# Plot the actual spectra
junk = spectra_w.line( xw,
                       yw, 
                      legend_label = "Cal Lamp Spectra", 
                        line_width = 3
                     )
#spectra_w.legend.location = 'bottom_right'
spectra_w.legend.location = 'top_left'
show(spectra_w)